In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import BertTokenizer, TFBertForSequenceClassification, AdamWeightDecay
import numpy as np
import tensorflow as tf

file_path = '/mnt/data/bot_detection_data.csv'
df = pd.read_csv(file_path)


texts = df['Tweet'].astype(str).values
labels = df['Bot Label'].values


texts_train, texts_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

print("Treinando Logistic Regression com TF-IDF...")

tfidf = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf.fit_transform(texts_train)
X_test_tfidf = tfidf.transform(texts_test)

lr_model = LogisticRegression(class_weight='balanced', max_iter=1000)
lr_model.fit(X_train_tfidf, y_train)

y_pred_lr = lr_model.predict(X_test_tfidf)

print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_lr))

auc_lr = roc_auc_score(y_test, y_pred_lr)
print(f'Logistic Regression AUC: {auc_lr}\n')

print("Treinando modelo BERT...")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(texts, max_len=128):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='tf'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    return tf.concat(input_ids, axis=0), tf.concat(attention_masks, axis=0)

input_ids_train, attention_masks_train = tokenize_data(texts_train)
input_ids_test, attention_masks_test = tokenize_data(texts_test)

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}
print("Class weights:", class_weights_dict)

bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

optimizer = AdamWeightDecay(learning_rate=2e-5)
bert_model.compile(optimizer=optimizer, loss=bert_model.compute_loss, metrics=['accuracy'])

history = bert_model.fit(
    [input_ids_train, attention_masks_train],
    y_train,
    validation_data=([input_ids_test, attention_masks_test], y_test),
    epochs=3,
    batch_size=16,
    class_weight=class_weights_dict
)

print("Avaliando o modelo BERT...")

test_logits = bert_model.predict([input_ids_test, attention_masks_test]).logits
y_pred_bert = tf.argmax(test_logits, axis=1)

print("BERT Classification Report:")
print(classification_report(y_test, y_pred_bert))

auc_bert = roc_auc_score(y_test, y_pred_bert)
print(f'BERT AUC: {auc_bert}')


Descrição das Etapas:
Carregamento e Pré-processamento dos Dados:

Objetivo: Carregar o dataset de tweets e dividir os dados entre treino e teste.
Descrição: Os textos dos tweets e seus respectivos rótulos (bots ou não bots) são extraídos do dataset. Em seguida, os dados são divididos em conjuntos de treino e teste usando o train_test_split, para garantir que possamos avaliar o modelo posteriormente em dados que ele não viu durante o treinamento.
Logistic Regression com TF-IDF:

Objetivo: Treinar um modelo mais simples de classificação (Logistic Regression) para comparação com o BERT.
Descrição: A etapa transforma os textos dos tweets em vetores de frequência de termos inversamente ponderados por frequência no documento (TF-IDF), que capturam a importância das palavras nos textos. Com esses vetores, o modelo Logistic Regression é treinado para classificar os tweets. A avaliação deste modelo é realizada usando métricas como precisão, recall, F1 Score e AUC, que medem a capacidade do modelo de identificar corretamente os bots.
Tokenização com BERT Tokenizer:

Objetivo: Preparar os dados para o modelo BERT.
Descrição: O BERT exige que os textos sejam convertidos em uma sequência de IDs de tokens, além de máscaras de atenção para lidar com padding. A função tokenizer.encode_plus é usada para tokenizar e gerar essas sequências e máscaras de atenção. Essas representações tokenizadas são usadas como entradas para o modelo BERT.
Rebalanceamento das Classes:

Objetivo: Ajustar o modelo para lidar com o desbalanceamento de classes (bots vs. não bots).
Descrição: O dataset pode ter um desbalanceamento, com mais exemplos de uma classe do que da outra (como mais tweets de bots do que de não bots, ou vice-versa). Para contornar isso, calculamos os pesos das classes com compute_class_weight, e esses pesos são aplicados durante o treinamento para garantir que o modelo não favoreça excessivamente a classe majoritária.
Treinamento do Modelo BERT:

Objetivo: Treinar um modelo BERT para a classificação de tweets em bots e não bots.
Descrição: Utilizando o modelo pré-treinado TFBertForSequenceClassification, treinamos o modelo BERT com os dados tokenizados. O modelo é ajustado com a técnica de AdamWeightDecay para otimização e os pesos de classe são aplicados para lidar com o desbalanceamento. O treinamento é realizado por 3 épocas, com validação nos dados de teste em cada época.
Avaliação do Modelo BERT:

Objetivo: Avaliar o desempenho do modelo BERT treinado.
Descrição: Após o treinamento, o modelo BERT é avaliado nos dados de teste. Utilizamos as previsões do modelo para calcular métricas de classificação, incluindo precisão (quantos bots foram corretamente identificados), recall (quantos bots o modelo conseguiu identificar), F1 Score (um equilíbrio entre precisão e recall), e AUC (que mede a capacidade do modelo de separar as classes).


##Análise dos Resultados:
Modelo Logistic Regression:

Desempenho: O modelo Logistic Regression, que é mais simples, serve como uma referência para o desempenho em relação ao BERT. Ele pode ter um desempenho razoável em conjuntos de dados menores ou com pouca complexidade linguística. Ao avaliar as métricas como precisão, recall e AUC, podemos ver como esse modelo se sai em relação ao modelo BERT.
Comparação com BERT: Se o modelo Logistic Regression tiver métricas comparáveis ou melhores que o BERT, pode sugerir que o BERT está subutilizando os dados ou que o problema pode ser resolvido com um modelo mais simples.
Modelo BERT:

Desempenho: O BERT é projetado para capturar nuances linguísticas e contextos complexos em textos, o que o torna ideal para tarefas como classificação de tweets. No entanto, devido à sua complexidade, ele pode exigir um ajuste mais cuidadoso (como o número de épocas, taxas de aprendizado e rebalanceamento de classes). A avaliação de precisão, recall e F1 Score nos ajuda a entender se o modelo está conseguindo distinguir corretamente entre bots e não bots.
Impacto do Rebalanceamento: O uso de pesos de classe durante o treinamento pode ter um impacto significativo, especialmente em datasets desbalanceados. Se o desempenho do modelo em detectar a classe minoritária (provavelmente "não bots") melhorar após o rebalanceamento, isso indicaria que o modelo está aprendendo a dar mais importância para essa classe.
AUC (Área Sob a Curva ROC):

AUC para Logistic Regression: O AUC mostra a capacidade do modelo Logistic Regression em separar as duas classes. Um AUC de 0,5 indicaria que o modelo não tem discriminação entre bots e não bots, enquanto um valor próximo de 1 indica uma separação perfeita.
AUC para BERT: Da mesma forma, a AUC para o BERT mostra sua capacidade de discriminar entre as duas classes. Como o BERT é um modelo mais sofisticado, esperamos que o AUC seja superior ao do modelo Logistic Regression, assumindo que o modelo BERT foi ajustado adequadamente.


##Próximos Passos:
Ajustes de Hiperparâmetros:

O número de épocas, o tamanho do lote, e a taxa de aprendizado podem ser ajustados para melhorar o desempenho do modelo BERT. Aumentar o número de épocas ou testar diferentes valores de taxa de aprendizado pode resultar em melhor desempenho.
Engenharia de Features:

Adicionar features adicionais além do texto pode melhorar o desempenho. Por exemplo, a contagem de seguidores, retweets, ou a presença de links no tweet podem ajudar o modelo a detectar padrões que distinguem bots de não bots.
Tuning dos Pesos das Classes:

Continuar experimentando com diferentes formas de balanceamento de classes pode ajudar a encontrar um ponto de equilíbrio que melhore tanto a precisão quanto o recall para as classes minoritárias.
Comparação com Outros Modelos:

Experimente outros modelos de linguagem pré-treinados, como RoBERTa ou DistilBERT, que podem ser mais rápidos ou mais eficientes que o BERT padrão, sem sacrificar tanto o desempenho.